In [ ]:
import requests
from bs4 import BeautifulSoup 

# 定义请求头
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# 基础URL
base_url = "http://changsha.gongjiao.com/xianlu_"

bus_lines = {} # 存储公交线路
stations = {} # 存储公交站点


In [ ]:
# 定义站点类
class Station:
    def __init__(self, name):
        self.name = name
        self.lines = set()
        self.neighbours = {}

In [ ]:
# 爬取所有线路
for num in range(28657, 28859):
    url = base_url + str(num)
    print(f"querying {url}")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 获取线路名
    line_num = soup.find('div', class_='gj01_lineSite_title').text.split()[0]
    line_num = line_num[:line_num.find('路') + 1]  

    # 如果有线路
    if line_num: 
        ul = soup.find('ul', class_='gj01_line_img') 
        bus_lines[line_num] = [a.text for a in ul.find_all('a', href=True)]
        for station in bus_lines[line_num]:
            if station not in stations:
                stations[station] = Station(station)
            stations[station].lines.add(line_num)


In [ ]:
# 添加邻居
for line, stops in bus_lines.items():
    for i, name in enumerate(stops):
        if i > 0:
            stations[name].neighbours[stops[i - 1]] = line
        if i < len(stops) - 1:
            stations[name].neighbours[stops[i + 1]] = line

In [ ]:
# Dijkstra算法计算最短路径
def dijkstra(start, end):
    # 初始化最短路径字典，键为节点，值为一个三元组（前驱节点，当前最短距离，当前线路）
    shortest_paths = {start: (None, 0, next(iter(stations[start].lines)))}
    current_node = start # 当前处理的节点
    current_line = shortest_paths[start][2] # 当前线路
    visited = set() # 已访问的节点集合

    while current_node != end: # 当当前节点不是终点时
        visited.add(current_node) # 将当前节点标记为已访问
        destinations = stations[current_node].neighbours # 获取当前节点的邻居节点及其对应的线路
        weight_to_current_node = shortest_paths[current_node][1] # 获取到当前节点的最短距离

        for next_node in destinations: # 遍历每一个邻居节点
            line = destinations[next_node] # 获取邻居节点的线路
            weight = 1 # 假设每个站点之间的距离为1
            total_weight = weight_to_current_node + weight # 计算从起点到达该邻居节点的总距离

            if next_node not in shortest_paths: # 如果邻居节点未被记录在最短路径字典中
                shortest_paths[next_node] = (current_node, total_weight, line) # 更新该邻居节点的最短路径信息
            else:
                current_shortest_weight = shortest_paths[next_node][1] # 获取当前记录的邻居节点的最短距离
                if current_shortest_weight > total_weight: # 如果新计算的距离更短，则更新该邻居节点的最短路径信息
                    shortest_paths[next_node] = (current_node, total_weight, line)

        next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited} # 筛选出所有未访问的节点
        if not next_destinations: # 如果没有未访问的节点，说明路径不可达
            return "Route Not Possible"
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1]) # 选择具有最短距离的未访问节点作为下一个当前节点

    path = [] # 构建最短路径，通过回溯前驱节点
    while current_node is not None:
        path.append(current_node)
        next_node, _, line = shortest_paths[current_node]
        if line != current_line and current_line is not None: # 如果线路发生变化，添加转乘信息
            path.append(f"转乘 {line}")
        current_line = line # 更新当前线路
        current_node = next_node # 回溯到前驱节点
    path = path[::-1] # 反转路径，使其从起点到终点
    return path, shortest_paths[end][1], shortest_paths[start][2] # 返回最短路径，路径总长度和起始线路

In [90]:
# 获取用户输入的起始站点和目标站点
print(f"可供选择的站点有：{', '.join(stations.keys())}\n")

可供选择的站点有：长沙火车站, 蓉园路口, 省公安厅, 袁家岭, 省军区, 清水塘(军区医院), 小吴门, 中山路, 先锋厅, 长沙轮渡, 橘子洲大桥东, 坡子街, 西湖桥, 楚湘街, 灵官渡, 第一师范, 大椿桥, 杏花园, 碧沙湖, 古堆山, 桔园小区, 华侨村, 桔园立交桥东(桔园路), 桔园立交桥北, 安贞医院(雨花亭北), 砂子塘, 省中医附一院(东塘南), 东塘西, 雅礼中学, 侯家塘, 长沙市三医院(仰天湖), 白沙路口, 沙河街, 南门口, 天心阁西门, 競才修业学校(柑子园西), 司门口, 贾谊故居, 解放西路口, 先锋厅(中山亭), 中山亭, 省中医院(巡道街), 省中医院(营盘街), 省妇幼, 湖南日报, 湘雅路口, 唐家巷, 芙蓉路口, 华夏, 华夏路口, 长雅中学, 黄兴北路秋月路口, 紫凤路, 北辰时代广场, 北辰三角洲, 劳动广场, 侯家塘南, 黄土岭, 省第二人民医院北(涂家冲), 涂家冲南, 长沙理工大学, 金盆岭, 公用客车厂, 省红十字妇幼医院(广厦新村), 猴子石路口, 百姓市场, 新开铺, 1103厂, 新开铺路友谊路口, 豹子岭, 二豹子岭, 湘府路大桥西, 洋湖湿地公园南门, 洋湖景园, 赤岗冲(多功能坪), 东塘东(长沙联通), 东塘北, 曹家坡, 长沙市妇幼保健院(长岭), 湖南省图书馆(窑岭北), 袁家岭南, 韭菜园, 牛耳教育(南阳街口), 华图教育(太平街口), 高叶塘, 省人防办东, 省人防办, 望麓桥, 麓山名园, 省结核病医院(省肺科医院), 教师村, 白鸽咀, 咸嘉新村, 咸嘉新村西, 湘麓山庄, 金星路口, 市政府, 市委, 观沙路, 含光路口[观沙路], 茶山路口, 长郡中学新校区, 曙光路口, 窑岭南, 铁道学院, 长沙市中心医院, 潇湘晨报, 林科大, 井坡子, 井湾子北, 井湾子南, 中建五局, 红星村南, 高升村南, 省植物园, 洞井铺, 汽车南站, 长沙火车南站, 香樟东路口, 长托, 圭塘, 龙骧巴士公司, 雨花区政府北, 窑坡, 师家老屋, 雨花区交警队, 新星小区, 雨花公安分局, 鼓风, 树木岭, 自然岭, 树木岭路口, 树木岭立交桥东, 矿通, 友谊新村, 狮子山东, 狮子山南, 红花坡, 茶园坡路口, 车站南路口, 茶园坡, 赤岗岭, 赤岗冲, 野坡, 长郡中学, 金霞苑总站,

In [92]:
start_station = input('起始站点名： ')
end_station = input('目标站点名：')
print()

# 计算最短路径
path, total_stops, first_line = dijkstra(start_station, end_station)
if path == "Route Not Possible":
    print(f"无法从 {start_station} 到 {end_station}")
else:
    print(f"总共需要经过 {total_stops} 站")
    print(f"首先乘坐的线路是 {first_line}")
    print(f"从 {start_station} 到 {end_station} 的最短路径为：")
    print(' -> '.join(path))


总共需要经过 37 站
首先乘坐的线路是 114路
从 长沙火车站 到 黄花机场首末站 的最短路径为：
转乘 114路 -> 长沙火车站 -> 转乘 805路 -> 人民路立交桥北 -> 转乘 317路 -> 德政园东 -> 芙蓉区政府 -> 人民路古曲路口 -> 东岸建材市场 -> 亚大南路口 -> 望龙村 -> 大汉建材批发城(红旗路口) -> 转乘 X107路 -> 红旗路人民东路口 -> 转乘 X202路 -> 大汉建材批发城(农科院) -> 转乘 X106路 -> 农科院北门 -> 转乘 114路 -> 司法警官学校 -> 泉塘西 -> 泉塘 -> 转乘 X102路 -> 砖厂 -> 泉塘安置区 -> 转乘 114路 -> 市看守所 -> 转乘 X101路 -> 长沙县一中 -> 黄兴大道口 -> 螺丝塘 -> 丁家村 -> 丁家村东 -> 丁家岭 -> 丁家粮库 -> 排头铺西 -> 排头铺 -> 观山村 -> 黄花新街 -> 长沙县六中 -> 黄花老街 -> 小塘安置区 -> 黄龙 -> 黄花工业园口 -> 机场口 -> 毛公塘 -> 黄花机场 -> 黄花机场首末站
